In [6]:
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

In [7]:
# store the task’s name, index, and prerequisite list
class Task:
    def __init__(self, name, index, prereq_numbers):
        self.name = name
        self.index = index
        self.prereq_numbers = prereq_numbers
    
    def __str__(self):
        return self.name
    
    def numbers_to_tasks(self, tasks):
        self.prereq_tasks = []
        for number in self.prereq_numbers:
            self.prereq_tasks.append(tasks[number])

In [8]:
class PoSorter:
    def __init__(self):
        self.tasks = []
        
    def verify_sort(self):
        #verify the sort is valid
        for i in range(len(self.sorted_tasks)):
            task = self.sorted_tasks[i]
            for prereq in prereq_tasks:
                if self.sorted_tasks.index(prereq) >= i:
                    return f'Task [{prereq}] does not come before [{task}]'
        return f'Successfully sorted {len(self.sorted_tasks)} out of {len(self.tasks)} tasks.
    
    def read_task(self, file):
        # read a task from the file
        # return a task or none if we reach the end of the file
        result = ''
        while len(result) == 0:
            line = file.readline()
            if len(line) == 0:
                return None
            line = line.strip()
            
            if len(line) > 0:
                fields = line.split(',', 2)
                index = int(fields[0])
                name = fields[1]
                prereq_text = fields[2].replace('[', '').replace(']', '').strip()
                
            if len(prereq_text) == 0:
                prereq_numbers = []
            else:
                prereq_numbers = list(map(int, prereq_text.split(',')))
            
            return Task(name, index, prereq_numbers)
    
    def load_po_file(self, filename):
        self.tasks = []
        file = open(filename, 'r')
        try:
            while True:
                new_task = self.read_task(file)
                if new_task == None:
                    break
                self.tasks.append(new_task)
        
        except Exception as e:
            massagebox.showinfo('Load Error', f'Error! {filename}' \n{e})
        
        finally:
            file.close()
            
        for task in self.tasks:
            task.numbers_to_tasks(self.tasks)
    
    def topo_sort(self):
        for task in self.tasks:
            task.followers = []
        
        for task in self.tasks:
            for prereq in task.prereq_tasks:
                prereq.followers.append(task)
            task.prereq_count = len(task.prereq_tasks)
        
        # Move tasks with no prerequisites onto the ready list.
        ready_tasks = []
        for task in self.tasks:
            if task.prereq_count == 0:
                ready_task.append(task)
        
        # sorted task list
        self.sorted_tasks = []
        
        while len(ready_tasks) > 0:
            ready_task = ready_tasks.pop(0)
            self.sorted_tasks.append(ready_task)
            
            for follower in ready_task.followers:
                follower.prereq_count -= 1
                if follower.prereq_count == 0:
                    ready_tasks.append(follower) # no prereqs, add it to the ready list

SyntaxError: EOL while scanning string literal (1471559723.py, line 12)

In [9]:
class App:
    def __init__(self):
        self.sorter = PoSorter()

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('topological_sorting')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('400x300')

        # Build the menu.
        self.menubar = tk.Menu(self.window)
        self.menu_file = tk.Menu(self.menubar, tearoff=False)
        self.menu_file.add_command(label='Open...', command=self.open_po, accelerator='Ctrl+O')
        self.menu_file.add_separator()
        self.menu_file.add_command(label='Exit', command=self.kill_callback)
        self.menubar.add_cascade(label='File', menu=self.menu_file)
        self.window.config(menu=self.menubar)

        # Build the item lists.
        frame = tk.Frame(self.window)
        frame.pack(padx=10, pady=(0, 10), side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        frame.columnconfigure(1, weight=1)
        frame.columnconfigure(3, weight=1)
        frame.rowconfigure(1, weight=1)

        # Unsorted list.
        inner_frame = tk.Frame(frame)
        inner_frame.grid(row=1, column=1, padx=3, pady=3, sticky='nsew')
        inner_frame.columnconfigure(1, weight=1)
        inner_frame.rowconfigure(1, weight=1)
        self.unordered_list = tk.Listbox(inner_frame)
        self.unordered_list.grid(row=1, column=1, sticky='nsew')
        scrollbar = tk.Scrollbar(inner_frame)
        scrollbar.grid(row=1, column=2, sticky='nse')
        self.unordered_list.config(yscrollcommand = scrollbar.set)
        scrollbar.config(command = self.unordered_list.yview)

        # Sort button.
        sort_button = tk.Button(frame, text='Sort', width=6, command=self.sort)
        sort_button.grid(row=1, column=2, padx=3)

        # Sorted list.
        inner_frame = tk.Frame(frame)
        inner_frame.grid(row=1, column=3, padx=3, pady=3, sticky='nsew')
        inner_frame.columnconfigure(1, weight=1)
        inner_frame.rowconfigure(1, weight=1)
        self.ordered_list = tk.Listbox(inner_frame)
        self.ordered_list.grid(row=1, column=1, sticky='nsew')
        scrollbar = tk.Scrollbar(inner_frame)
        scrollbar.grid(row=1, column=2, sticky='nse')
        self.ordered_list.config(yscrollcommand = scrollbar.set)
        scrollbar.config(command = self.ordered_list.yview)

        self.window.bind('<Control-o>', self.ctrl_o_pressed)

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def kill_callback(self):
        self.window.destroy()

    def ctrl_o_pressed(self, event):
        self.open_po()
        
    def open_po(self):
        file_types = [('Partial Ordering', '*.po')]
        filename = filedialog.askopenfilename(
            defaultextension='.po', filetypes=file_types,
            initialdir='.', title='Open Partial Ordering')
        if not filename: return

        self.unordered_list.delete(0,'end')
        self.ordered_list.delete(0,'end')
        self.sorter.load_po_file(filename)
        self.unordered_list.insert('end', *self.sorter.tasks)

    def sort(self):
        self.sorter.topo_sort()
        self.ordered_list.insert('end', *self.sorter.sorted_tasks)
        messagebox.showinfo('Sort Result',
            self.sorter.verify_sort())

In [10]:
App()